In [1]:
# disable overly verbose tensorflow logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}   
import tensorflow as tf

import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_curve, auc

## Import model of interest (change cell below)

In [2]:
##### leave the coming 3 lines alone
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
input = Input(input_shape)

##### change things in here:
### folder locations
notebook_folder_location = r"C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments"
image_location = r'C:\Users\20192735\Documents\8P361_images'

### model lines
model_name = 'Xception'
from tensorflow.keras.applications.xception import Xception, preprocess_input
pretrained = Xception(input_shape=input_shape, include_top=False, weights='imagenet')



####already lines for other model
#model_name = 'MobileNetV2'
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
#pretrained = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

In [3]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

    # dataset parameters
    train_path = os.path.join(base_dir, 'train+val', 'train')
    valid_path = os.path.join(base_dir, 'train+val', 'valid')

    # instantiate data generators
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary',
                                               shuffle=False)

    val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary',
                                             shuffle=False)

    return train_gen, val_gen

## Import pretrained model

In [4]:
output = pretrained(input)
output = GlobalAveragePooling2D()(output)
output = Dropout(0.5)(output)
output = Dense(1, activation='sigmoid')(output)

model = Model(input, output)

# note the lower lr compared to the cnn example
model.compile(SGD(learning_rate=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

# print a summary of the model on screen
model.summary()

# get the data generators
train_gen, val_gen = get_pcam_generators(image_location)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
____________________________________________

## Save model and weights (change cell below)

In [5]:
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json)
    
# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]

## Train the model

In [6]:
# train the model, note that we define "mini-epochs"
train_steps = train_gen.n//train_gen.batch_size//20
val_steps = val_gen.n//val_gen.batch_size//20

# since the model is trained for only 10 "mini-epochs", i.e. half of the data is
# not used during training
history = model.fit(train_gen, steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=20,    
                    shuffle=False,
                    callbacks=callbacks_list)


Epoch 1/20


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\20192735\Anaconda3\envs\8P361\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\20192735\AppData\Local\Temp/ipykernel_6092/3851132551.py", line 7, in <module>
    history = model.fit(train_gen, steps_per_epoch=train_steps,
  File "C:\Users\20192735\Anaconda3\envs\8P361\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\20192735\Anaconda3\envs\8P361\lib\site-packages\keras\engine\training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\20192735\Anaconda3\envs\8P361\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\20192735\Anaconda3\envs\8P361\lib\site-packages\tensorflow\python\eager\def_function.py", line 915, in __call__
    result = self._cal

TypeError: object of type 'NoneType' has no len()

In [7]:
val_loss=min(history.history['val_loss'])
loss=min(history.history['loss'])
accuracy=min(history.history['accuracy'])
val_accuracy=min(history.history['val_accuracy'])

NameError: name 'history' is not defined

## ROC analysis

In [ ]:
import matplotlib.pyplot as plt

model = tf.keras.models.load_model(notebook_folder_location+r"/"+weights_filepath)
y_pred = model.predict(val_gen).ravel()
y_true = val_gen.labels

fpr, tpr, thresholds = roc_curve(y_true, y_pred)
from sklearn.metrics import auc
auc = auc(fpr, tpr)



plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve '+model_name)
plt.legend(loc='best')

plt.savefig("ROC_curve_"+model_name+"_val_loss_"+str(round(val_loss,4))+"_val_accuracy_"+str(round(val_accuracy,4))+"_loss_"+str(round(loss,4))+"_accuracy_"+str(round(accuracy,4))+'.jpeg')
plt.show()

In [ ]:
import winsound
winsound.Beep(1000,1000)